# Importing libraries

In [331]:
import pandas as pd
import time
from datetime import datetime
import ydata_profiling
from pandas_profiling import ProfileReport
import numpy as np
import re
import requests
from selenium.webdriver.common.by import By
from selenium import webdriver
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException

# Reading CSV

In [332]:
df = pd.read_csv('data/stage_01_scraped_mediamarkt.csv')

# DF Report

In [333]:
profile = ProfileReport(df, title='Data Profiling Report', explorative=True)
profile

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

# Pandas

In [334]:
#overview
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 364 entries, 0 to 363
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   brand           364 non-null    object 
 1   model           364 non-null    object 
 2   category        364 non-null    object 
 3   size            355 non-null    float64
 4   storage         357 non-null    object 
 5   color           364 non-null    object 
 6   price           364 non-null    float64
 7   source          364 non-null    object 
 8   webpage         364 non-null    object 
 9   condition       364 non-null    object 
 10  date            364 non-null    object 
 11  article_number  364 non-null    int64  
 12  n_of_reviews    364 non-null    object 
 13  rating          255 non-null    float64
 14  delivery_time   364 non-null    object 
dtypes: float64(3), int64(1), object(11)
memory usage: 42.8+ KB


,size,price,article_number,rating
count,355.000000,364.000000,3.640000e+02,255.000000
mean,6.447777,650.321429,2.176201e+06,4.487451
std,0.428290,394.031598,5.054629e+04,0.699324
min,4.700000,49.700000,1.971824e+06,1.000000
25%,6.100000,345.950000,2.151048e+06,4.400000
50%,6.600000,619.000000,2.193028e+06,4.700000
75%,6.700000,899.000000,2.216176e+06,5.000000
max,7.600000,1899.000000,2.242341e+06,5.000000


### Looking for duplicates

In [335]:
df.duplicated().sum()  # Find number of duplicates

0

### Missing Values

In [336]:
df.isnull().sum() 

brand               0
model               0
category            0
size                9
storage             7
color               0
price               0
source              0
webpage             0
condition           0
date                0
article_number      0
n_of_reviews        0
rating            109
delivery_time       0
dtype: int64

### Unique values

In [337]:
df.nunique()

brand              17
model             110
category            3
size               33
storage             8
color             124
price             192
source              1
webpage           364
condition           1
date                1
article_number    364
n_of_reviews       63
rating             22
delivery_time      10
dtype: int64

# Manipulating Target Variabeles

Target variables are the columns that are important for the end structure

### Brand

In [338]:
df['brand'].unique()

array(['APPLE', 'XIAOMI', 'SAMSUNG', 'NOTHING', 'MOTOROLA', 'DORO',
       'GOOGLE', 'FAIRPHONE', 'EMPORIA', 'RUGGEAR', 'NOKIA', 'INOI',
       'OPPO', 'CROSSCALL', 'ONE', 'WIKO', 'PEAQ'], dtype=object)

In [339]:
df['brand'] = df['brand'].str.lower()

In [340]:
df['brand'].isnull().sum() 

0

In [341]:
valid_brands = ['apple', 'xiaomi', 'samsung', 'nothing', 'motorola', 'fairphone', 
                'google', 'doro', 'inoi', 'emporia', 'one', 'nokia', 'ruggear', 
                'oppo', 'crosscall', 'wiko', 'peaq', 'huawei', 'lg', 'sony', 'htc', 
                'oneplus', 'zte', 'alcatel', 'asus', 'blackberry', 'realme', 'vivo', 
                'tecno', 'lenovo', 'meizu', 'honor', 'ulefone', 'cat']

In [342]:
def validate_brand(brand):
    if brand not in valid_brands:
        print(f"Unrecognized brand, please verify: {brand}")

# Apply the function to the 'brand' column
df['brand'].apply(validate_brand)

0      None
1      None
2      None
3      None
4      None
       ... 
359    None
360    None
361    None
362    None
363    None
Name: brand, Length: 364, dtype: object

### Model

In [343]:
df['model'].unique()

array(['iPhone 15 ', 'Redmi A2 ', 'iPhone 15 Pro ', 'iPhone 15 Pro Max ',
       'iPhone 14 ', 'iPhone 13 ', 'iPhone 11 (2020) ', 'Redmi 13C ',
       'iPhone SE (2022) ', 'Galaxy A55 5G ', 'Galaxy A15 4G ',
       'iPhone SE ', 'Galaxy S23+ ', 'Galaxy Z Flip5 ', 'Galaxy A54 5G ',
       'phone (2a) ', 'Galaxy S23 ', 'Redmi Note 13 Pro+ 5G ',
       'Galaxy S24 Ultra ', 'Redmi Note 13 5G ', 'Galaxy S24+ ',
       'iPhone 15 Plus ', 'phone (2) ', 'iPhone 14 Plus ', 'Redmi 9A ',
       'Galaxy A25 5G ', 'Redmi Note 13 4G ', 'iPhone 12 ', 'Galaxy S24 ',
       '14 + Electric Scooter 4 Lite Swiss Edition Bundle ',
       'Galaxy A34 5G ', 'Moto G13 ', 'Moto G54 5G ', 'Galaxy S23 Ultra ',
       'Galaxy A35 5G ', '6820 ', 'Galaxy S23 FE ', 'Pixel 8 Pro ',
       'XCover6 Pro Enterprise Edition ', '5 5G ',
       'Galaxy XCover 5 Enterprise Edition ', 'Redmi Note 13 Pro 4G ',
       'Redmi 12 ', 'TOUCHsmart.2 ', '13T ', 'RG360 ',
       'Redmi Note 13 Pro 5G ', 'G42 5G ', 'Pixel 8 ', 'JOY',


In [344]:
df['model'] = df['model'].str.lower()

In [345]:
def strip(model):
    return model.rstrip()

df['model'] = df['model'].apply(strip)

In [346]:
df['model'].isnull().sum() 

0

### Category

In [347]:
# Only smartphones are the interest of the scrapping
df['category'].unique()

array(['Smartphone', 'Klapphandy', 'Mobiltelefon'], dtype=object)

In [348]:
# Filtering unecessary categories

In [349]:
df = df[df['category'] == 'Smartphone']

In [350]:
df['category'] = df['category'].str.lower()

/var/folders/g4/6_vmpgcj1f54bd8m6rxjg2nh0000gp/T/ipykernel_15290/842447539.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['category'] = df['category'].str.lower()


In [351]:
df['category'].isnull().sum() 

0

### Size

In [398]:
df['size'].unique()

array([6.1  , 6.52 , 6.7  , 6.74 , 4.7  , 6.6  , 6.5  , 6.4  , 6.67 ,
       6.8  , 6.53 , 6.2  , 6.36 , 6.46 , 5.3  , 6.79 , 3.25 , 3.   ,
       6.56 , 7.6  , 6.95 , 6.73 , 6.72 , 6.497, 6.71 , 4.95 , 6.517,
       5.   , 6.3  , 5.45 , 6.43 , 6.55 , 6.9  , 6.58 , 6.08 , 6.39 ])

In [392]:
df_size_nan = df[np.isnan(df['size'])]
print(len(df_size_nan))
df_size_nan

TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [395]:
driver = webdriver.Firefox()
wait = WebDriverWait(driver, 10) 

# Assuming size_verification is a dictionary with URLs to check
display_sizes = {}
size_verification = df_size_nan['webpage']

for index, url in size_verification.items():
    try:
        driver.get(url)  # Navigate to each URL
        wait.until(EC.presence_of_element_located((By.ID, 'features')))  # Wait for the features section to load

        display_size_in_inches = driver.execute_script("""
            var features = document.querySelector('#features');
            var sections = features.querySelectorAll('section');
            for (var i = 0; i < sections.length; i++) {
                var h2 = sections[i].querySelector('h2');
                if (h2 && h2.textContent.trim().toLowerCase() === 'display') {
                    var dts = sections[i].querySelectorAll('dt');
                    for (var j = 0; j < dts.length; j++) {
                        if (dts[j].textContent.trim() === 'Bildschirmdiagonale (Zoll):') {
                            var dd = dts[j].nextElementSibling;
                            if (dd) {
                                return dd.textContent.replace('"', '').trim();
                            }
                            return null;
                        }
                    }
                }
            }
            return 'Display section or data not found';
            """)

        display_sizes[url] = display_size_in_inches
        

    except TimeoutException:
        print(f"Timed out waiting for features section at URL {url}")
    except Exception as e:
        print(f"Error processing URL {url}: {e}")

# Close the WebDriver after the loop
driver.quit()

In [396]:
for url, size in display_sizes.items():
    df.loc[df['webpage'] == url, 'size'] = size
    

In [397]:
df['size'] = df['size'].astype(float)

/var/folders/g4/6_vmpgcj1f54bd8m6rxjg2nh0000gp/T/ipykernel_15290/3343612739.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['size'] = df['size'].astype(float)


,brand,model,category,size,storage,color,price,source,webpage,condition,date,article_number,n_of_reviews,rating,delivery_time
144,emporia,touchsmart.2,smartphone,NaN,NaN,EMPORIA TOUCHsmart.2 - Smartphone (Schwarz/Silber,115.95,mediamarkt,https://www.mediamarkt.ch/de/product/_emporia-...,new,2024-04-15,2110132,(8),2.5,None
148,ruggear,rg360,smartphone,NaN,8 GB,Schwarz,121.95,mediamarkt,https://www.mediamarkt.ch/de/product/_ruggear-...,new,2024-04-15,2053578,(0),NaN,None
261,wiko,y52,smartphone,NaN,16 GB,Deep Blue,49.70,mediamarkt,https://www.mediamarkt.ch/de/product/_wiko-y52...,new,2024-04-15,2127285,(2),1.5,None


### Storage

In [376]:
df['storage'].unique()

array(['128 GB', '32 GB', '256 GB', '64 GB', '512 GB', '1 TB', '8 GB',
       '16 GB'], dtype=object)

In [377]:
df_space_nan = df[pd.isnull(df['storage'])]
storage_verification = df_space_nan['webpage']

In [386]:
driver = webdriver.Firefox()
wait = WebDriverWait(driver, 10) 

# Assuming size_verification is a dictionary with URLs to check
storages = {}
storage_verification = df_space_nan['webpage']

for index, url in storage_verification.items():
    try:
        driver.get(url)  # Navigate to each URL
        wait.until(EC.presence_of_element_located((By.ID, 'features')))  # Wait for the features section to load

        storagevalues = driver.execute_script("""
            var features = document.getElementById('features');
            if (!features) {
                return 'Features section not found';
            }
            var sections = features.querySelectorAll('section');
            for (var i = 0; i < sections.length; i++) {
                var h2 = sections[i].querySelector('h2');
                if (h2 && h2.textContent.trim().toLowerCase() === 'technische merkmale') {
                    var dts = sections[i].querySelectorAll('dt');
                    for (var j = 0; j < dts.length; j++) {
                        if (dts[j].textContent.trim() === 'Speicherkapazität:') {
                            var dd = dts[j].nextElementSibling;
                            return dd ? dd.textContent.trim() : null;
                        }
                    }
                }
            }
            return 'Display section or data not found';
        """)

        storages[url] = storagevalues
        

    except TimeoutException:
        print(f"Timed out waiting for features section at URL {url}")
    except Exception as e:
        print(f"Error processing URL {url}: {e}")

# Close the WebDriver after the loop
driver.quit()

In [375]:
for url, storage in storages.items():
    df.loc[df['webpage'] == url, 'storage'] = size

### Color

In [254]:
df['color'].unique()
#df['color'].unique()

df['color'] = df['color'].str.lower()

colors = ['black', 'blue', 'green', 'red', 'yellow', 'white', 'gray', 'purple', 'pink', 'orange', 
          'brown', 'silver', 'gold', 'titanium', 'platinum', 'schwarz', 'weiss']

def extract_color(value):
    # Check if the string contains numbers, 'GB', or specific special characters
    if re.search(r'\d|GB|[()/]', value):
        # Convert the value to lowercase to make the search case-insensitive
        value_lower = value.lower()

        # Search for each color in the string
        for color in colors:
            if re.search(r'\b' + color + r'\b', value_lower):
                return color  # Return the color with the first letter capitalized

        # Return 'Unknown' or any other placeholder if no known color is found
        return 'unknown'
    
    # If the string doesn't contain the specified patterns, return it as is
    return value

df['color'] = df['color'].apply(extract_color)
#df['color'].unique()

df_color_nan = df[pd.isnull(df['color'])]

/var/folders/g4/6_vmpgcj1f54bd8m6rxjg2nh0000gp/T/ipykernel_15290/4101708950.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['color'] = df['color'].str.lower()
/var/folders/g4/6_vmpgcj1f54bd8m6rxjg2nh0000gp/T/ipykernel_15290/4101708950.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['color'] = df['color'].apply(extract_color)


In [255]:
df['color'] = df['color'].apply(extract_color)
df['color'].unique()

/var/folders/g4/6_vmpgcj1f54bd8m6rxjg2nh0000gp/T/ipykernel_15290/1274586712.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['color'] = df['color'].apply(extract_color)


array(['black', 'schwarz', 'black titanium', 'yellow', 'midnight',
       'natural titanium', 'blue', 'starlight', 'white titanium', 'green',
       'purple', 'midnight black', 'pink', 'awesome lilac',
       'blue titanium', 'blue black', 'phantom black', 'awesome navy',
       'graphite', 'awesome lime', 'weiss', 'awesome white',
       'titanium black', 'white', 'graphite black', 'onyx black',
       'awesome iceblue', 'grau', 'cream', 'awesome graphite',
       'arctic white', 'titanium violet', 'titanium grey',
       'awesome lemon', 'granite grey', 'clover green', 'mint green',
       'cobalt violet', 'jade green', 'awesome silver', 'hellgrün',
       'matte charcoal', 'midnight blue', 'red', 'marble grey',
       'titanium yellow', 'navy blue', 'lavender', 'obsidian',
       'matte black', 'transparent edition', 'forest green',
       'amber yellow', 'so grey', 'icy blue', 'mint', 'meteor grey',
       'lagoon blue', 'alpine blue', 'mineral grey', 'cosmic black',
       'violet

In [256]:
color_translations = {
    'schwarz': 'black', 'weiss': 'white', 'grau': 'gray',
    'hellgrün': 'light green', 'hellblau': 'light blue', 'violett': 'violet',
    'dunkelblau': 'dark blue', 'blau': 'blue', 'graphit': 'graphite',
    'rot': 'red', 'grün': 'green', 'gelb': 'yellow',
    'orange': 'orange', 'rosa': 'pink', 'lila': 'purple',
    'braun': 'brown', 'beige': 'beige', 'türkis': 'turquoise',
    'gold': 'gold', 'silber': 'silver', 'kupfer': 'copper',
    'marine': 'navy', 'oliv': 'olive', 'khaki': 'khaki',
    'karmesin': 'crimson', 'fuchsia': 'fuchsia', 'aquamarin': 'aquamarine',
    'koralle': 'coral', 'indigo': 'indigo', 'elfenbein': 'ivory',
    'lavendel': 'lavender', 'limette': 'lime', 'magenta': 'magenta',
    'maroon': 'maroon', 'ocker': 'ochre', 'pfirsich': 'peach',
    'pflaume': 'plum', 'saphir': 'sapphire', 'smaragd': 'emerald',
    'sonne': 'sun', 'taupe': 'taupe', 'teal': 'teal',
    'zimt': 'cinnamon', 'zitrone': 'lemon'
}

In [283]:
df['color'] = df['color'].replace(color_translation)

In [284]:
df_color_nan = df[pd.isnull(df['color'])]
df_color_nan

,brand,model,size,storage,color,rating,n_of_reviews,delivery_time,price,webpage,source,date


### Price

In [285]:
df['price'].unique()

array([ 719.  ,   79.95, 1029.  , 1129.  ,  619.  ,  525.  ,  399.  ,
        816.  ,  118.95,  929.  ,  497.95,  383.95,  479.95,  169.  ,
        829.  ,  430.95, 1349.  ,  699.  ,  479.  ,  844.  ,  347.95,
        329.95,  629.  ,  413.95,  429.95,  339.7 , 1179.  ,  254.95,
        819.  ,  915.  ,  849.  ,  691.  ,  379.95,  378.95,  549.  ,
        429.  , 1118.  ,  726.  ,  381.95,  263.95, 1242.  , 1399.  ,
        133.95,  269.95,  729.  ,  169.95,  749.  ,  949.  , 1099.  ,
       1556.  ,  309.7 ,   88.95,  129.95,  176.95, 1563.  , 1260.  ,
        621.  ,  997.  , 1149.  ,  359.  ,  359.95,  998.  ,  126.95,
        921.  ,  412.95,  887.  , 1599.  ,  847.  ,  767.  ,  152.95,
        610.  ,  206.  ,  642.  ,  649.  ,  296.95,  139.95, 1459.  ,
        912.  ,  419.95,  389.95,  606.  ,  115.95,  551.  , 1026.  ,
        121.95,  349.95,  899.  ,  567.  ,  529.  , 1456.  , 1253.  ,
        203.95,  652.  , 1474.  ,  669.  ,  867.  ,  222.95, 1770.  ,
       1003.  ,  159

In [286]:
def clean_price(value):
    try:
        # Convert the value to string in case it's not
        value = str(value)
        # Find all numeric sequences
        matches = re.findall(r'\d+\.?\d*', value)
        if matches:
            # Return the last match as a float
            return float(matches[-1])
    except Exception as e:
        # Log the error and return the value as is or return NaN
        print(f"Error cleaning price for value {value}: {e}")

df['price'] = df['price'].apply(clean_price)

In [287]:
df['price'] = df['price'].astype(float)

### Source

In [288]:
df_source_nan = df[pd.isnull(df['webpage'])]

In [289]:
"""
def check_url_status(url):
    try:
        response = requests.head(url, timeout=2)  # Using HEAD instead of GET to speed up the process
        if response.status_code == 200:
            return 'Working'
        else:
            return f'Broken ({response.status_code})'
    except requests.RequestException as e:
        return f'Error ({e})'

# Apply the function to check each URL
df['status'] = df['webpage'].apply(check_url_status)
"""

"\ndef check_url_status(url):\n    try:\n        response = requests.head(url, timeout=2)  # Using HEAD instead of GET to speed up the process\n        if response.status_code == 200:\n            return 'Working'\n        else:\n            return f'Broken ({response.status_code})'\n    except requests.RequestException as e:\n        return f'Error ({e})'\n\n# Apply the function to check each URL\ndf['status'] = df['source'].apply(check_url_status)\n"

### Condition

In [290]:
# no necessary manipulation

### Date

In [291]:
df['date'] = pd.to_datetime(df['date'])

### n_of_reviews

In [384]:
df['n_of_reviews'].unique()

array([ 24,  10,  15,  13,   0,  12,  32,   8,  34,  67,   1,   4,   3,
        11,   9,  50,   7,  18,  78,  17,   5, 155,  28,  57, 320,   2,
         6, 272, 318, 882,  16, 210, 274,  56, 326, 881,  19, 295, 216,
        21, 271, 613, 604, 607, 605,  14, 154,  22, 273, 211,  26,  49,
       296, 148, 336, 156,  55,  48, 319,  44,  25,  36,  35])

In [385]:
df_n_of_reviews_nan = df[pd.isnull(df['n_of_reviews'])]
len(df_n_of_reviews_nan)

0

In [381]:
n_of_reviews_nan_verification = df_n_of_reviews_nan['webpage']
for index, url in n_of_reviews_nan_verification.items():
    print(f"{index}: {url}")

In [382]:
reviews_manually = {38: '(155)'}
for index, reviews in reviews_manually.items():
    df.at[index, 'n_of_reviews'] = reviews

In [383]:
df['n_of_reviews'] = df['n_of_reviews'].str.replace('(', '', regex=False).str.replace(')', '', regex=False)
df['n_of_reviews'] = df['n_of_reviews'].fillna('0').astype(int)

/var/folders/g4/6_vmpgcj1f54bd8m6rxjg2nh0000gp/T/ipykernel_15290/3398413562.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['n_of_reviews'] = df['n_of_reviews'].str.replace('(', '', regex=False).str.replace(')', '', regex=False)
/var/folders/g4/6_vmpgcj1f54bd8m6rxjg2nh0000gp/T/ipykernel_15290/3398413562.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['n_of_reviews'] = df['n_of_reviews'].fillna('0').astype(int)


### Rating

In [297]:
df['rating'].unique()

array([4.8, 4.1, 4.9, 0. , 4.6, 5. , 4.7, 4.3, 4.5, 4.4, 4. , 3.3, 4.2,
       3.7, 3. , 2.5, 1. , 2. , 3.8, 1.5, 2.3, 3.5, 3.4])

In [298]:
df_rating_nan = df[pd.isnull(df['rating'])]
len(df_rating_nan)
df_rating_nan

,brand,model,size,storage,color,rating,n_of_reviews,delivery_time,price,webpage,source,date


In [299]:
rating_nan_verification = df_rating_nan['webpage']
for index, url in rating_nan_verification.items():
    print(f"{index}: {url}")

In [300]:
df['rating'] = df['rating'].fillna('0').astype(float)

### Delivery Time

In [303]:
df['delivery_time'].unique()

array([ 2.,  6., nan,  7.,  4., 10.])

In [307]:
def extract_days(text):
    # Convert text to string to handle cases where text is not a string
    text = str(text)
    
    if pd.isnull(text) or "nicht mehr verfügbar" in text or "nicht lieferbar" in text or "ausverkauft" in text or "kein Liefertermin" in text:
        return None
    else:
        # Find all numbers in the string
        numbers = [int(num) for num in re.findall(r'\d+', text)]
        if numbers:
            return max(numbers)  # Return the highest number, assuming it's the upper limit of days
        else:
            return None  # Return None if no numbers are found

# Assuming you have a DataFrame 'df' with a column 'delivery_time'
df['delivery_time'] = df['delivery_time'].apply(extract_days)


In [308]:
df['delivery_time'].astype(float)
df['delivery_time'].info

<bound method Series.info of 0      2.0
1      2.0
2      2.0
3      2.0
4      2.0
      ... 
359    NaN
360    NaN
361    2.0
362    NaN
363    4.0
Name: delivery_time, Length: 358, dtype: float64>

In [309]:
df

,brand,model,size,storage,color,rating,n_of_reviews,delivery_time,price,webpage,source,date
0,apple,iphone 15,6.10,128 GB,black,4.8,0,2.0,719.00,https://www.mediamarkt.ch/de/product/_apple-ip...,mediamarkt,2024-04-15
1,xiaomi,redmi a2,6.52,32 GB,black,4.1,0,2.0,79.95,https://www.mediamarkt.ch/de/product/_xiaomi-r...,mediamarkt,2024-04-15
2,apple,iphone 15 pro,6.10,256 GB,black titanium,4.9,0,2.0,1029.00,https://www.mediamarkt.ch/de/product/_apple-ip...,mediamarkt,2024-04-15
3,apple,iphone 15 pro max,6.70,256 GB,black titanium,4.9,0,2.0,1129.00,https://www.mediamarkt.ch/de/product/_apple-ip...,mediamarkt,2024-04-15
4,apple,iphone 15,6.10,128 GB,yellow,0.0,0,2.0,719.00,https://www.mediamarkt.ch/de/product/_apple-ip...,mediamarkt,2024-04-15
...,...,...,...,...,...,...,...,...,...,...,...,...
359,apple,iphone 15 plus,6.70,512 GB,pink,0.0,0,NaN,1159.00,https://www.mediamarkt.ch/de/product/_apple-ip...,mediamarkt,2024-04-15
360,xiaomi,redmi note 12 pro 5g,6.67,128 GB,midnight black,4.6,0,NaN,239.70,https://www.mediamarkt.ch/de/product/_xiaomi-r...,mediamarkt,2024-04-15
361,apple,iphone 12,6.10,64 GB,white,4.4,0,2.0,479.95,https://www.mediamarkt.ch/de/product/_apple-ip...,mediamarkt,2024-04-15
362,apple,iphone 14 plus,6.70,256 GB,blue,5.0,0,NaN,829.00,https://www.mediamarkt.ch/de/product/_apple-ip...,mediamarkt,2024-04-15


In [280]:
df = df.drop(columns=['category', 'article_number', 'condition' ])

In [281]:
new_order = ['brand', 'model', 'size', 'storage', 'color', 'rating', 'n_of_reviews', 'delivery_time', 'price', 'webpage', 'source', 'date']
df = df[new_order]

In [282]:
file_name = "data/stage02_cleaned_mediamarkt.csv"
# Save the DataFrame to CSV in the same directory as the script
df.to_csv(file_name, index=False)